In [2]:
## LIBRARIES IMPORT FOR TROPIDASH APP ##
 
from eccodes import *
from datetime import datetime, timedelta

import requests
import pandas as pd
import numpy as np
import ipywidgets as widgets

from IPython.display import Javascript, display, clear_output

import warnings
warnings.filterwarnings("ignore")

from tracks_utils import *
from atm_utils import * 
from impacts_utils import *
from utils_TemporalEvolution import *

In [3]:
## WIDGET TO IMPORT THE TITLE IMAGE ##

file = open("layout_data/title.jpg", "rb")
image = file.read()
title_img = widgets.Image(
    value=image,
    format='jpg',
)

In [4]:
## WIDGET TO DEFINE THE STARTING DATE OF THE FORECAST ##
## This sets the date of which we want to download the forecast ##

start_date_forecast = widgets.DatePicker(
    description = 'Forecast date:',
    value = datetime(datetime.now().year, datetime.now().month, datetime.now().day, hour=0),
)

start_date_forecast.style.description_width = '90px'

In [4]:
## DOWNLOAD THE FORECAST TRACKS ##
# The forecast data is saved as bufr file called tc_test_track_data.bufr    
start_date = download_tracks_forecast(start_date_forecast.value)

# Boolean to remember if the forecast is from yesterday for eventual warnings in the app
if start_date != start_date_forecast.value:
    yesterday_forecast = True
else:
    yesterday_forecast = False

## DOWNLOAD THE OBSERVED TRACKS DATA FROM IBTrACS ##
# The observed data is saved as a csv file called ibtracs.ACTIVE.list.v04r00.csv
url = 'https://www.ncei.noaa.gov/data/international-best-track-archive-for-climate-stewardship-ibtracs/v04r00/access/csv/ibtracs.ACTIVE.list.v04r00.csv'
r = requests.get(url, allow_redirects=True)
save_file = 'data/tracks/ibtracs.ACTIVE.list.v04r00.csv'
with open(save_file, 'wb') as f:
    f.write(r.content)

In [5]:
## LOAD THE FORECAST DATA OF THE TRACKS IN A DATAFRAME ##
df_storms_forecast = create_storms_df(start_date)

## LOAD THE OBSERVED TRACKS IN A DATAFRAME ## 
df_storms_observed = pd.read_csv('data/tracks/ibtracs.ACTIVE.list.v04r00.csv', header=[0,1])

## CREATE THE LIST OF ACTIVE CYCLONES IN THE FORECAST ##
cycl = np.array((df_storms_forecast.stormIdentifier.unique(), df_storms_forecast.longStormName.unique()))
active_cyclones = [f"{cycl[0,c]}-{cycl[1,c].lstrip()}" for c in range(cycl.shape[1])]

In [6]:
## WIDGET TO SHOW THE LIST OF ACTIVE CYCLONES AND SELECT THE ONE OF WHICH WE WANT TO PLOT THE DATA ##

cyclone = widgets.Dropdown(
    options = active_cyclones,
    description = 'Active Storms:',
    disabled=False,
)
cyclone.style.description_width = '90px'
cyclone.style.font_weight = 'bold'

# Update storms list when downloading new tracks data
def update_storms_list(_):
    start_date = download_tracks_forecast(start_date_forecast.value)
    df_storms_forecast = create_storms_df(start_date)
    cycl = np.array((df_storms_forecast.stormIdentifier.unique(), df_storms_forecast.longStormName.unique()))
    active_cyclones = [f"{cycl[0,c]}-{cycl[1,c].lstrip()}" for c in range(cycl.shape[1])]
    cyclone.options = active_cyclones

start_date_forecast.observe(update_storms_list, names='value')

# Print widget
message_cyclone = widgets.HTML(
    value = "<center>After selecting the date please click the dropdown menu to check if there are active storms in the forecast.</center>"
    "<center>The list may take a few seconds to update.</center>",
)

In [7]:
## WIDGET TO SELECT WHICH ENSEMBLE MEMBERS OF THE FORECAST TRACK PLOT ##

members = df_storms_forecast.ensembleMemberNumber.unique().tolist()
ens_members = widgets.SelectMultiple(
    options=members,
    value=[],
    description="Ensemble members to plot:",
    disable=False
)

ens_members.style.description_width = '168px'
ens_members.style.font_weight = 'bold'

# Update ensemble members list when new track data are available
def update_ensembles_list(_):
    start_date = download_tracks_forecast(start_date_forecast.value)
    df_storms_forecast = create_storms_df(start_date)
    ens_members.options = df_storms_forecast.ensembleMemberNumber.unique().tolist()
    
cyclone.observe(update_ensembles_list, names='value')
    
# Print widget
message_ens = widgets.HTML(
    value="Multiple values can be selected with <b>shift</b> and/or <b>ctrl</b> (or <b>command</b>)",
)

In [8]:
## WIDGETS FOR RETURN PERIOD SELECTION ##

vec_rp_coh = ["5yr", "10yr", "50yr", "100yr", "250yr", "500yr", "1000yr"]
widget_sel_rp_coh = widgets.Dropdown(options = vec_rp_coh, description = 'Return Period Coastal Hazard')
widget_sel_rp_coh.style.description_width = "180px"

vec_rp_cyh = ["50yr", "100yr", "250yr", "500yr", "1000yr"]
widget_sel_rp_cyh = widgets.Dropdown(options = vec_rp_cyh, description = 'Return Period Cyclone Hazard')
widget_sel_rp_cyh.style.description_width = "180px"

In [9]:
## UPDATE BUTTON TO PLOT NEW TRACKS AND DATA DEPENDING ON THE CYCLONE SELECTED ##

update_button = widgets.Button(
    description = 'Update Forecast',
    icon = "check",
    style=dict(
        button_color='blue',
        font_weight='bold',
        text_color='lightgreen',
        text_decoration='underline',
))

update_output = widgets.Output()

## Function to create a html widget that insert a title for each section of the app ##

def create_section_title(title):
    title = widgets.HTML(
        value=f"<h1>{title}</h1>",
        placeholder='',
        description='',
    )
    return title

## Plot new data when clicking the Update Forecast Button ##
def on_button_clicked(b):
    with update_output:
        ## Print message if the forecast has no active cyclones ##
        if len(cyclone.options) == 0:
            display(widgets.HTML(value=f"There are no active cyclones in the forecast of <b>{start_date_forecast.value.strftime('%d %b %Y')}</b>"), clear=True)
            display(widgets.HTML(value=f"Please select another date."))
        else:
            ## Display Section 1 Title
            display(create_section_title("Section 1 - Tropical Cyclone Tracks and Characteristics"), clear=True)
            ## Output of initial lat-lon, average track and initial-final time step of the cyclone for the next sections ##
            code, name = cyclone.value.split('-')
            display(widgets.HTML(value=f"Creating the dataset for tropical cyclone <b>{name}</b>")) # clear previous cyclones plots
            df_storms_forecast = create_storms_df(start_date_forecast.value)
            df_storms_observed = pd.read_csv('data/tracks/ibtracs.ACTIVE.list.v04r00.csv', header=[0,1])
            df_f = df_storms_forecast[df_storms_forecast.stormIdentifier == code]
            df_f.reset_index(drop=True, inplace=True)
            df_o = df_storms_observed[df_storms_observed.NAME.squeeze() == name]
            df_o.reset_index(drop=True, inplace=True)
            # Initial and final lat-lon
            initial_lat_lon = (df_f.latitude.iloc[0], df_f.longitude.iloc[0])
            final_lat_lon = (df_f.latitude.iloc[-1], df_f.longitude.iloc[-1])
            coord = ((initial_lat_lon[0] - final_lat_lon[0])/2, (initial_lat_lon[1] - final_lat_lon[1])/2)
            # Average track
            avg_info = mean_forecast_track(df_f)
            locations_avg = avg_info[0]
            # Initial and final time step
            df_f["date"] = datetime(df_f.year[0], df_f.month[0], df_f.day[0], df_f.hour[0]) + timedelta(hours=1) * df_f.timePeriod
            # If we select a forecast date in the past the cyclone might not be present in the observations data because we can access only yhe latest one
            if len(df_o) > 0:
                initial_timestep = datetime.strptime(df_o.ISO_TIME.squeeze()[0], "%Y-%m-%d %H:%M:%S")
            else:
                initial_timestep = start_date_forecast.value
                display(widgets.HTML(value=f"Cyclone <b>{name}</b> is not present in the latest observations data of IBTrACS"))
            final_timestep = df_f.date.iloc[-1].to_pydatetime()
            display(widgets.HTML(value="Dataset prepared!"))

            ## Plots of the tracks ## 
            display(widgets.HTML(value=f"Creating the plots for the tropical cyclone tracks considering ensemble members: <b>{ens_members.value}</b>"))
            tc_track_map = plot_cyclone_tracks_ipyleaflet(ens_members.value, df_f, df_o)
            display(tc_track_map)

            ## Plot of atmopsheric variables ##
            display(create_section_title("Section 2 - Atmospheric Variables"))
            variables = ['msl', '2t', 'tp', 'wind']
            stepsdict = {
                "base": [24, 48, 120, 240],
                "10fgg15": ["0-24", "24-48", "96-120", "216-240"]
                }
            fnames = dwnl_atmdata_step(variables, stepsdict, start_date_forecast.value)
            vardict = load_atmdata(variables, fnames)
            vec_forecast = [f"{x}h from today" for x in stepsdict["base"]]
            widget_sel_forecast = widgets.Dropdown(options = vec_forecast, description = 'Forecast')
            def update_plot(select_forecast):
                tooldict = {
                    "24h from today": 0,
                    "48h from today": 1,
                    "120h from today": 2,
                    "240h from today": 3
                }
                s = tooldict[select_forecast]
                
                m = plot_atmdata_step(vardict, s, coord, stepsdict)
                
                display(m)
            display(widgets.interactive(update_plot, select_forecast = widget_sel_forecast))

            ## Plots of impact variables ## 
            display(create_section_title("Section 3 - Impact Variables"))
            def update_plot2(rp_coh, rp_cyh):
                #Download
                print("Dwonloading data...")
                dwnl_coastalhaz(rp_coh)
                dwnl_cyclonehaz(rp_cyh)
                dwnl_riskidx()
            
                #Load
                print("Loading data...")
                coh = load_coastalhaz(rp_coh, open = True)
                cyh = load_cyclonehaz(rp_cyh, open = True)
            
                #Plot
                m = Map(center = coord, zoom = 3)
                print("Adding Coastal Hazard")
                m = plot_coastalhaz(coh, rp_coh, m = m)
                print("Adding Cyclone Hazard")
                m = plot_cyclonehaz(cyh, rp_cyh, m = m)
                print("Adding Population")
                m = plot_poplayer(m = m)
                print("Adding Exposition Indexes")
                m = plot_riskidx(["Tsunamis", "Coastal_floods", "Sea_level_rise"], m = m)
                m.add_control(LayersControl())
                m.layout.height = "700px"
            
                display(m)
            display(widgets.interactive(update_plot2, rp_coh = widget_sel_rp_coh, rp_cyh = widget_sel_rp_cyh))

            ## Plots of section 5 ##
            # create function and write it there
            
            ## Map of temporal evolution ##
            display(create_section_title("Section 5 - Temporal Evolution"))
            # maps5 = map_s5(initial_lat_lon, intial_timestep.strftime("%Y%m%d"), final_timestep.strftime("%Y%m%d"), locations_avg)
            maps5 = map_s5(initial_lat_lon, start_date_forecast.value.strftime("%Y%m%d"), final_timestep.strftime("%Y%m%d"), 'data/df_avg_track.csv')
            display(maps5)

update_button.on_click(on_button_clicked)

In [8]:
## WIDGET FOR INFOMATION BUTTON ##

info_button = widgets.Button(
    description = 'TropiDash Info',
    icon = "info",
    style=dict(
        button_color='orange',
        font_weight='bold',
        text_color='black',
        text_decoration='underline',
        border='solid black 1px',
))

notify_output = widgets.Output()
update_info = widgets.Output()
display(notify_output)

@notify_output.capture()
def popup(text):
    clear_output()
    display(Javascript("alert('{}')".format(text)))

def clickme(b):
    info_message = "Information regarding the Dashboard are displayed below the graphs."
    popup(info_message)
    with update_info:
        display(widgets.HTML(value="<center><h1>Tropi<font color='orange'>Dash</font> Information</h1></center>"), clear=True)
        display(widgets.HTML(value="TropiDash is an interactive dashboard to visualize and consult tropical cyclone data. It is composed of 5 sections:"
                             "<ol type='1'>" "<li>Tropical Cyclone Tracks and Characteristics</li>" 
                             "<li>Atmospheric Variables</li>" 
                             "<li>Impact Variables</li>" 
                             "<li>Multi-Layer Map combining data from all previous sections</li>" 
                             "<li>Point-Wise Temporal Evolution of Atmospheric Variables</li> </ol>"))
        display(widgets.HTML(value="All documents regarding the dashboard are available at the following <a href='https://github.com/ECMWFCode4Earth/TropiDash' target='_blank'><font color='blue'><u>github repository</u></font></a>."))
        display(widgets.HTML(value="You can find more information about each section in the <a href='https://github.com/ECMWFCode4Earth/TropiDash/blob/main/TropiDash_documentation.md' target='_blank'><font color='blue'><u>dashboard documentation</u></font></a>."))
        display(widgets.HTML(value="For specific tutorial examples please refer to the folder <a href='https://github.com/ECMWFCode4Earth/TropiDash/tree/main/tutorials' target='_blank'><font color='blue'><u>tutorials</u></font></a>"))
        display(widgets.HTML(value="To use the dashboard, select the forecast date, the active cyclone, the numbers of ensembles to plot and click the <img src='layout_data/update_button.jpg' width='70'> button."
                             " The dashboard will automatically update the plots and the data displayed in the sections."))
        display(widgets.HTML(value="If you want to install the dashboard locally, please find the required packages to run the notebook in the file <a href='https://github.com/ECMWFCode4Earth/TropiDash/blob/main/requirements.yml' target='_blank'><font color='blue'><u>requirements.yml</u></font></a>."))
        display(widgets.HTML(value="For any questions or feedback please contact the dashboard developers:"
                             "<ul> <li> Filippo Dainelli <a href='mailto:filippo.dainelli@polimi.it'><font color='blue'><u>send email</u></font></a></li>"
                             "<li> Paolo Colombo <a href='mailto:paolo1.colombo@polimi.it'><font color='blue'><u>send email</u></font></a></li>"
                             "<li> Laura Paredes i Fortuny <a href='mailto:paredes_laufor@externos.gva.es <paredes_laufor@externos.gva.es>'><font color='blue'><u>send email</u></font></a></li></ul>"))
        display(widgets.HTML(value="<center><h3>Thank you for using Tropi<font color='orange'>Dash</font>!</h3></center>"))
info_button.on_click(clickme)

Output()

In [10]:
## GROUP THE THREE PRINCIPAL WIDGET IN A VERTICAL BOX STRUCTURE ##
if yesterday_forecast:
    forecast_day_message = widgets.HTML(value=f"<center><b><font color='red'>WARNING:</font></b> The forecast from <b>{start_date_forecast.value.strftime('%d %b %Y')}</b> is not available yet.</center>" 
                                        f"<center><b>{start_date.strftime('%d %b %Y')}</b> will be used instead, please select this date to process the dataset accordingly.</center>")
    warning_box = widgets.Box([forecast_day_message],
                    layout=widgets.Layout(
                        display='flex',
                        border='3px dashed red',
                        align_items='center',
                        width='100%'
                    ))
    group_box = widgets.Box([forecast_day_message, message_cyclone, cyclone, message_ens, ens_members],
                  layout=widgets.Layout(
                      display='flex',
                      flex_flow='column',
                      border='3px dashed orange',
                      align_items='center',
                      width='100%'
                  ))
    selection_box = widgets.Box([warning_box, group_box],
                    layout=widgets.Layout(
                        display='flex',
                        flex_flow='column',
                        align_items='center',
                        width='100%'
                    ))
else:
    selection_box = widgets.Box([start_date_forecast, message_cyclone, cyclone, message_ens, ens_members],
                    layout=widgets.Layout(
                        display='flex',
                        flex_flow='column',
                        border='3px dashed orange',
                        align_items='center',
                        width='100%'
                    ))

## GROUP THE THREE PRINCIPAL WIDGET AND THE UPDATE BUTTON TOGETHER
widgets_box = widgets.Box([selection_box, update_button],
                          layout=widgets.Layout(
                              display='flex',
                              flex_flow='column',
                              align_items='center',
                              # align_content='space-between',
                              # width='100%',
                              # height='100%'
                          ))

## GROUP THE TITLE IMAGE WITH THE INFORMATION BUTTON ##
info_box = widgets.Box([title_img, info_button],
                        layout=widgets.Layout(
                            display='flex',
                            flex_flow='column',
                            align_items='center',
                        ))


## GROUP THE TITLE IMAGE WITH THE GENERAL WIDGETS BOX ##
title_box = widgets.Box([info_box, widgets_box],
                        layout=widgets.Layout(
                            display='flex',
                            flex_flow='row',
                            align_items='center'
                        ))

## GROUP THE WIDGETS, THE GRAPHS AND THE INFORMATION DISPLAY ## 
display_box = widgets.Box([title_box, update_output, update_info],
                          layout=widgets.Layout(
                                display='flex',
                                flex_flow='column',
                                align_items='center',
                          ))

display(display_box)

Box(children=(Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x01\x00\xcd\x00\xcd\x00\x00\xff\xe2\x0f\x…

Output()